In [1]:
# | default_exp solvemodels

## Setup

In [2]:
# | export


import json
import sys

import cobra

# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.buildingediting
import mmon_gcm.solving
import mmon_gcm.supermodel
import pandas as pd

In [3]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is where to adjust whether fva is run if running in notebook
args = {
    "run_fva": "True",
    "no_processes": 4,
    "output_dir": "../outputs/model_solutions/",
    "model_file": "../models/4_stage_GC.xml",
    "map_file": "../inputs/map.json",
    "parameters_file": "../inputs/arabidopsis_parameters.csv",
}

sys.argv = ["script_name"] + list(args.values())

In [4]:
# | export

run_fva = sys.argv[1]
if run_fva == "True":
    run_fva = True
elif run_fva == "False":
    run_fva = False
else:
    raise ValueError(f"Please specify True or False for run_fva, not {run_fva}")

no_processes = int(sys.argv[2])
output_dir = sys.argv[3]
model_file = sys.argv[4]
map_file = sys.argv[5]
parameters_file = sys.argv[6]

## Instantiate and apply base constraints to model

### Import FBA model

In [5]:
# | export

four_stage_GC_model = cobra.io.sbml.read_sbml_model(model_file)  # read model

No objective coefficients in model. Unclear what should be optimized


Decrease tolerance

In [6]:
# | export

print(four_stage_GC_model.solver.configuration.tolerances.integrality)
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)
four_stage_GC_model.solver.configuration.tolerances.feasibility = 1e-7
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-07


### Define reactions to use for FVA

In [7]:
# | export

if run_fva == True:
    linker_list = [
        reaction
        for reaction in four_stage_GC_model.reactions
        if "gc_Linker_2" in reaction.id
        or "gc_Linker_1" in reaction.id
        or "_total_pseudolinker_1" in reaction.id
        or "_total_pseudolinker_2" in reaction.id
        or "ae_gc" in reaction.id
    ]

    extra_fva_rxns = set()

    with open(map_file, "r+") as f:
        map_data = json.load(f)

    for reaction in map_data[1]["reactions"].values():
        extra_fva_rxns.add(four_stage_GC_model.reactions.get_by_id(reaction["bigg_id"]))

    fva_list = list(set(linker_list) | extra_fva_rxns)

else:
    fva_list = []

### Instantiate Supermodel

Import parameters from csv, created in [parameters notebook](1.2_defining_parameters.ipynb)

In [8]:
# | export

parameters_df = pd.read_csv(parameters_file, index_col=0)
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
R_ch_vol,2.004760e-01,Dimensionless,Knoblauch et al. (2023)
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)


In [9]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(parameters_df.loc[:, "Value"], fba_model=four_stage_GC_model)

In [10]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True);

Volume in phase 0 is 0.38pL
Volume in phase 1 is 0.4375pL
Volume in phase 2 is 0.38pL
Volume in phase 3 is 0.38pL


In [11]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True, per_guard_cell=False);

Volume in phase 0 is 0.0002204dm3
Volume in phase 1 is 0.00025375dm3
Volume in phase 2 is 0.0002204dm3
Volume in phase 3 is 0.0002204dm3


This is a little different to Jezek and Blatt (2017), where they have closed as 0.3-0.4pL and open as 0.5-0.65. Open apertures are on the small side but still within their range (2-6). Wang used a volume of 0.4pL.

### Constrain SuperModel

#### Constrain osmolarity of the model using the equation from the OnGuard model (Hills et al 2012)

In [12]:
# | export

arabidopsis_supermodel.constrain_osmolarity(printouts=True);

Raw osmolarities: [0.039359327616195004, 0.05492236409486689, 0.039359327616195004, 0.039359327616195004]
Osmolarities in mM: [178.58134127 216.44281417 178.58134127 178.58134127]
Change in osmolarity: 37.86147289566091mM
c osmolarities constrained to [0.00980047 0.01367567 0.00980047 0.00980047]
v osmolarities constrained to [0.02955886 0.0412467  0.02955886 0.02955886]


In [13]:
import numpy as np

In [14]:
np.array(arabidopsis_supermodel.get_osmolarities())

array([0.03935933, 0.05492236, 0.03935933, 0.03935933])

In [15]:
help(arabidopsis_supermodel.get_osmolarities)

Help on method get_osmolarities in module mmon_gcm.supermodel:

get_osmolarities(apertures='default', equation='onguard', printouts=False) method of mmon_gcm.supermodel.SuperModel instance
    Calculates the osmolarities at each phase of the model using the equation
    specified, default is onguard. Adds these as an attribute to the SuperModel



In [16]:
mmoles_per_m2 = np.array(arabidopsis_supermodel.get_osmolarities())
mmoles_per_gc = mmoles_per_m2 / arabidopsis_supermodel.N_gcs
moles_per_gc = mmoles_per_gc * 10**-3
fmoles_per_gc = moles_per_gc / 10**-15
print(f"Closed fmoles: {fmoles_per_gc[0]}")
print(f"Open fmoles: {fmoles_per_gc[1]}")
print(f"Increase in fmoles: {fmoles_per_gc[1] - fmoles_per_gc[0]}")

Closed fmoles: 67.86090968309483
Open fmoles: 94.69373119804636
Increase in fmoles: 26.832821514951533


#### Constrain photons using a PPFD of 150µmolm$^{-2}$s$^{-1}$, same as used in Horrer et al (2016)

In [17]:
# | export

PPFD = 150
arabidopsis_supermodel.constrain_photons(PPFD, printouts=True);

Total leaf volume: 0.17dm3
Guard cell volume in 1m2 leaf: 0.0003dm3
Mesophyll cell volume in 1m2 leaf: 0.091dm3
Proportion of the leaf that is gc: 0.003
PPFD 150umolphotonsm-2
Photon influx into leaf: 486.0 mmolphotonsm-2hr-1
e = 0.012
Photon influx into Guard cells: 0.018mmolphotonsm-2hr-1
Photon influx into mesophyll cells: 485.982mmolphotonsm-2hr-1


## Add maintenance constraint to model

gc_ratio is the ratio of gc vs me maintenance obtained from 2.0_compute_maintenance

In [18]:
# | export

arabidopsis_supermodel.add_maintenance(gc=True, gc_ratio=4.427086760474383e-05);

### Test blocking Complex III in blue light WT

In [34]:
arabidopsis_supermodel.fba_model.reactions.get_by_id("SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mc_gc_2").bounds = 0, 0
arabidopsis_supermodel.fba_model.reactions.get_by_id("SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2").bounds = 0, 0

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (no_CIII, no_CIII_sol) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-20 18:29:37.018698
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-06-20 18:29:48.834174, that took 0.19692460695902506 minutes


In [33]:
arabidopsis_supermodel.fba_model.reactions.get_by_id("SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mc_gc_2").bounds = 0, cobra.Configuration().upper_bound
arabidopsis_supermodel.fba_model.reactions.get_by_id("SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2").bounds = 0, cobra.Configuration().upper_bound

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (CIII, CIII_sol) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-20 18:29:25.458722
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-06-20 18:29:37.003048, that took 0.19240544239679971 minutes


In [35]:
CIII_sol.loc['Phloem_tx_overall'].loc['fluxes']

14.25882479432091

In [36]:
no_CIII_sol.loc['Phloem_tx_overall'].loc['fluxes']

14.258823306193605

Phloem_tx_overall: 14.258823306193605 without Complex III; 14.25882479432091 with Complex III

### Test for maltose degradation pathways

In [38]:
arabidopsis_supermodel.fba_model.reactions.get_by_id("RXN_2141_p_gc_2").bounds = 0, 0
arabidopsis_supermodel.fba_model.reactions.get_by_id("RXN_2141_p_gc_2").bounds = 0, 0

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (maltose_c, maltose_c_sol) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-20 21:03:53.851992
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-06-20 21:04:05.724912, that took 0.19788201252619425 minutes


In [37]:
arabidopsis_supermodel.fba_model.reactions.get_by_id("RXN_2141_p_gc_2").bounds = 0, cobra.Configuration().upper_bound
arabidopsis_supermodel.fba_model.reactions.get_by_id("RXN_2141_p_gc_2").bounds = 0, cobra.Configuration().upper_bound

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (maltose_pc, maltose_pc_sol) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-20 21:03:40.588376
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-06-20 21:03:53.784066, that took 0.21992816925048828 minutes


In [41]:
maltose_pc_sol.loc[['Phloem_tx_overall', 'RXN_2141_p_gc_2', 'MALTODEG_RXN_c_gc_2']]['fluxes']

Phloem_tx_overall      1.425882e+01
RXN_2141_p_gc_2        5.259830e-03
MALTODEG_RXN_c_gc_2   -2.062496e-13
Name: fluxes, dtype: float64

In [43]:
maltose_pc_sol.loc['Phloem_tx_overall'].loc['fluxes']

14.258823306193605

In [42]:
maltose_c_sol.loc[['Phloem_tx_overall', 'RXN_2141_p_gc_2', 'MALTODEG_RXN_c_gc_2']]['fluxes']

Phloem_tx_overall      14.258821
RXN_2141_p_gc_2         0.000000
MALTODEG_RXN_c_gc_2     0.006945
Name: fluxes, dtype: float64

In [44]:
maltose_c_sol.loc['Phloem_tx_overall'].loc['fluxes']

14.258821433564357

In [47]:
(maltose_pc_sol.loc['Phloem_tx_overall'].loc['fluxes']-maltose_c_sol.loc['Phloem_tx_overall'].loc['fluxes'])/maltose_pc_sol.loc['Phloem_tx_overall'].loc['fluxes']

1.3133126120572098e-07

In [50]:
maltose_c_sol.loc[['GLC_c_gc_Linker_2', 'MALTOSE_c_gc_Linker_2', 'GLC_c_gc_Linker_1', 'MALTOSE_c_gc_Linker_1']]['fluxes']

GLC_c_gc_Linker_2        0.003473
MALTOSE_c_gc_Linker_2    0.006744
GLC_c_gc_Linker_1        0.000000
MALTOSE_c_gc_Linker_1    0.006744
Name: fluxes, dtype: float64

In [53]:
print(maltose_c_sol[maltose_c_sol.index.str.contains("gc_2") & (maltose_c_sol['fluxes'].abs() > 0.0000001)].sort_values(by='fluxes').to_string())

                                             fluxes  minimum  maximum
Pi_PROTON_mc_gc_2                         -0.010418      NaN      NaN
H2O_pc_gc_2                               -0.007739      NaN      NaN
EX_X_O2_t_gc_2                            -0.001810      NaN      NaN
CO2_tx_gc_2                               -0.001810      NaN      NaN
CO2_ec_gc_2                               -0.001810      NaN      NaN
ISOCITDEH_RXN_c_gc_2                      -0.001736      NaN      NaN
OAA_MAL_pc_gc_2                           -0.001736      NaN      NaN
O2_mc_gc_2                                -0.001736      NaN      NaN
PGLUCISOM_RXN_p_gc_2                      -0.001207      NaN      NaN
X5P_Pi_pc_gc_2                            -0.000868      NaN      NaN
RIB5PISOM_RXN_p_gc_2                      -0.000603      NaN      NaN
1TRANSKETO_RXN_p_gc_2                     -0.000603      NaN      NaN
F16ALDOLASE_RXN_c_gc_2                    -0.000302      NaN      NaN
2_PERIOD_7_PERIOD_1_

# Test blocking hexose accumulation - See maltose accumulation

In [19]:
#for i in range(4):
    #arabidopsis_supermodel.fba_model.reactions.get_by_id("GLC_c_gc_Linker_" + str(i+1)).bounds = 0, 0
    #arabidopsis_supermodel.fba_model.reactions.get_by_id("GLC_v_gc_Linker_" + str(i+1)).bounds = 0, 0
    #arabidopsis_supermodel.fba_model.reactions.get_by_id("FRU_c_gc_Linker_" + str(i+1)).bounds = 0, 0
    #arabidopsis_supermodel.fba_model.reactions.get_by_id("FRU_v_gc_Linker_" + str(i+1)).bounds = 0, 0

## Blue Light

### ATPase unconstrained

#### WT

In [20]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)
 
    (
        blue_unconstrained_wt,
        blue_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 15:56:45.113653
Running pFBA
Running FVA on 350 reactions using 4 processes
MAL_c_gc_Linker_1
Cl_v_gc_Linker_1
aCIT_v_gc_Linker_2
LYS_v_gc_Linker_2
FRU_c_gc_Linker_1
GDPKIN_RXN_c_gc_2
GLC_ae_gc_2
FRU_c_gc_Linker_2
CIT_v_gc_Linker_2
TYR_v_gc_Linker_1
GLC_c_gc_Linker_2
NITRATE_c_gc_Linker_1
TYR_v_gc_Linker_2
aCIT_v_gc_Linker_1
aMAL_v_gc_Linker_1
RXN_1461_v_gc_2
SUC_CIT_mc_gc_2
SUCROSE_PROTON_cv_gc_2
F16ALDOLASE_RXN_c_gc_2
GLC_v_gc_Linker_1
NITRATE_PROTON_rev_vc_gc_2
FRU_total_pseudolinker_2
2KG_OAA_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
FRU_total_pseudolinker_1
HIS_v_gc_Linker_2
FRU_PROTON_ec_gc_2
ARG_v_gc_Linker_2
NITRATE_ae_gc_1
Pi_ae_gc_1
AMMONIUM_ae_gc_1
HIS_v_gc_Linker_1
RXN_14351_pc_gc_2
SUC_ISOCITRATE_mc_gc_2
NITRATE_ae_gc_2
Pi_ae_gc_2
AMMONIUM_ae_gc_2
MAL_SO4_mc_gc_2
GLC_v_gc_Linker_2
PALMITATE_c_gc_Linker_1
THF_pc_gc_2
CO2_ec_gc_2
NITRATE_ae_gc_3
Pi_ae_gc_3
AMMONIUM_ae_gc_3
ACONITATEHYDR_RXN_c_gc_2
PALMITATE_c_gc_Linker_2
3PGAREARR_RXN_c_gc_

RXN490_3650_p_gc_2
2TRANSKETO_RXN_p_gc_2
Cl_ae_gc_3
CYS_v_gc_Linker_1
OAA_MAL_pc_gc_2
6PGLUCONOLACT_RXN_p_gc_2
2KG_SUC_mc_gc_2
CYS_v_gc_Linker_2
FRU_v_gc_Linker_1
PEPCARBOX_RXN_c_gc_2
2_KETOGLUTARATE_xc_gc_2
SUC_Pi_mc_gc_2
PHE_v_gc_Linker_1
MAL_total_pseudolinker_1
6PGLUCONDEHYDROG_RXN_p_gc_2
SUCROSE_ae_gc_1
SULFATE_ae_gc_1
Cl_ae_gc_1
SUCROSE_ae_gc_4
SULFATE_ae_gc_4
Cl_ae_gc_4
MAL_total_pseudolinker_2
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi_gc_2
F16BDEPHOS_RXN_p_gc_2
PHE_v_gc_Linker_2
FRU_v_gc_Linker_2
ADENYL_KIN_RXN_p_gc_2
SUCROSE_ae_gc_2
Cl_ae_gc_2
SULFATE_ae_gc_2
FRUCTOKINASE_RXN_c_gc_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
ATP_ADP_Pi_pc_gc_2
SUC_FUM_mc_gc_2
PSII_RXN_p_gc_2
UDPKIN_RXN_c_gc_2
MAL_ACONITATE_mc_gc_2
RXN_10773_c_gc_2
1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_2
GLC_pc_gc_2
CIT_c_gc_Linker_1
H2O_mc_gc_2
2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2
Photon_tx_gc_2
aMAL_ae_gc_1
CIT_c_gc_Linker_2
K_total_pseudolinker_1
aMAL_ae_gc_2
H2O_ec_gc_2
K_total_pseudolinker_2
PROTON_ATPase_c_gc_2
GLY_pc

In [21]:
# | export

blue_unconstrained_wt_solution.to_csv(output_dir + f"blue_unconstrained_wt.csv")

#### Starch KO

In [22]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        blue_unconstrained_starchko,
        blue_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 16:06:42.590114
Running pFBA
Running FVA on 350 reactions using 4 processes
MAL_c_gc_Linker_1
Cl_v_gc_Linker_1
aCIT_v_gc_Linker_2
LYS_v_gc_Linker_2
FRU_c_gc_Linker_1
GDPKIN_RXN_c_gc_2
GLC_ae_gc_2
FRU_c_gc_Linker_2
CIT_v_gc_Linker_2
TYR_v_gc_Linker_1
GLC_c_gc_Linker_2
NITRATE_c_gc_Linker_1
TYR_v_gc_Linker_2
aCIT_v_gc_Linker_1
aMAL_v_gc_Linker_1
RXN_1461_v_gc_2
SUC_CIT_mc_gc_2
SUCROSE_PROTON_cv_gc_2
F16ALDOLASE_RXN_c_gc_2
GLC_v_gc_Linker_1
NITRATE_PROTON_rev_vc_gc_2
FRU_total_pseudolinker_2
2KG_OAA_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
FRU_total_pseudolinker_1
HIS_v_gc_Linker_2
FRU_PROTON_ec_gc_2
ARG_v_gc_Linker_2
NITRATE_ae_gc_1
Pi_ae_gc_1
AMMONIUM_ae_gc_1
HIS_v_gc_Linker_1
RXN_14351_pc_gc_2
SUC_ISOCITRATE_mc_gc_2
NITRATE_ae_gc_2
Pi_ae_gc_2
AMMONIUM_ae_gc_2
MAL_SO4_mc_gc_2
GLC_v_gc_Linker_2
PALMITATE_c_gc_Linker_1
THF_pc_gc_2
CO2_ec_gc_2
NITRATE_ae_gc_3
Pi_ae_gc_3
AMMONIUM_ae_gc_3
ACONITATEHYDR_RXN_c_gc_2
PALMITATE_c_gc_Linker_2
3PGAREARR_RXN_c_gc_

RXN490_3650_p_gc_2
2TRANSKETO_RXN_p_gc_2
Cl_ae_gc_3
CYS_v_gc_Linker_1
OAA_MAL_pc_gc_2
6PGLUCONOLACT_RXN_p_gc_2
2KG_SUC_mc_gc_2
CYS_v_gc_Linker_2
FRU_v_gc_Linker_1
PEPCARBOX_RXN_c_gc_2
2_KETOGLUTARATE_xc_gc_2
SUC_Pi_mc_gc_2
PHE_v_gc_Linker_1
MAL_total_pseudolinker_1
6PGLUCONDEHYDROG_RXN_p_gc_2
SUCROSE_ae_gc_1
SULFATE_ae_gc_1
Cl_ae_gc_1
SUCROSE_ae_gc_4
SULFATE_ae_gc_4
Cl_ae_gc_4
MAL_total_pseudolinker_2
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi_gc_2
F16BDEPHOS_RXN_p_gc_2
PHE_v_gc_Linker_2
FRU_v_gc_Linker_2
ADENYL_KIN_RXN_p_gc_2
SUCROSE_ae_gc_2
Cl_ae_gc_2
SULFATE_ae_gc_2
FRUCTOKINASE_RXN_c_gc_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
ATP_ADP_Pi_pc_gc_2
SUC_FUM_mc_gc_2
PSII_RXN_p_gc_2
UDPKIN_RXN_c_gc_2
MAL_ACONITATE_mc_gc_2
RXN_10773_c_gc_2
1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_2
GLC_pc_gc_2
CIT_c_gc_Linker_1
H2O_mc_gc_2
2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2
Photon_tx_gc_2
aMAL_ae_gc_1
CIT_c_gc_Linker_2
K_total_pseudolinker_1
aMAL_ae_gc_2
H2O_ec_gc_2
K_total_pseudolinker_2
PROTON_ATPase_c_gc_2
GLY_pc

In [23]:
# | export

blue_unconstrained_starchko_solution.to_csv(output_dir + "blue_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [ ]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        blue_constrained_wt,
        blue_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 17:46:59.842055
Running pFBA
Running FVA on 350 reactions using 4 processes
RXN_1461_v_gc_2
RXN490_3650_p_gc_2
TYR_v_gc_Linker_2
K_PROTON_cv_gc_2
SUCROSE_ae_gc_3
GLC_ae_gc_3
SULFATE_ae_gc_3
Cl_ae_gc_3
GLT_v_gc_Linker_1
CIT_v_gc_Linker_1
FRU_total_pseudolinker_1
MAL_c_gc_Linker_2
Cl_v_gc_Linker_2
aCIT_v_gc_Linker_2
OAA_MAL_pc_gc_2
RXN_14351_pc_gc_2
SUCROSE_ae_gc_4
GLC_ae_gc_4
SULFATE_ae_gc_4
Cl_ae_gc_4
FRU_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLT_v_gc_Linker_2
CIT_v_gc_Linker_2
6PGLUCONDEHYDROG_RXN_c_gc_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
CO2_tx_gc_2
H2O_ec_gc_2
NADPH_Dehydrogenase_p_gc_2
ADENYL_KIN_RXN_c_gc_2
GLC_pc_gc_2
OAA_Pi_mc_gc_2
MGII_ae_gc_1
Pi_ae_gc_1
AMMONIUM_ae_gc_1
aMAL_ae_gc_1
L_ASPARTATE_v_gc_Linker_1
K_total_pseudolinker_1
RIB5PISOM_RXN_p_gc_2
MGII_ae_gc_2
Pi_ae_gc_2
AMMONIUM_ae_gc_2
K_total_pseudolinker_2
aMAL_ae_gc_2
TRIOSEPISOMERIZATION_RXN_p_gc_2
WATER_ae_gc_1
CAII_ae_gc_1
K_ae_gc_1
MAL_ae_gc_1
L_ASPARTATE_v_gc_Linker_2

In [ ]:
# | export

blue_constrained_wt_solution.to_csv(output_dir + "blue_constrained_wt.csv")

#### Starch KO

In [ ]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        blue_constrained_starchko,
        blue_constrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

blue_constrained_starchko_solution.to_csv(output_dir + "blue_constrained_starchko.csv")

## White Light

### Unconstrained ATPase

#### WT

In [ ]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        white_unconstrained_wt,
        white_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

white_unconstrained_wt_solution.to_csv(output_dir + "white_unconstrained_wt.csv")

#### Starch KO

In [ ]:
 # | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        white_unconstrained_starchko,
        white_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

white_unconstrained_starchko_solution.to_csv(output_dir + "white_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [19]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        white_constrained_wt,
        white_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 20:37:59.746821
Running pFBA
Running FVA on 350 reactions using 4 processes
ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1

NITRATE_ae_gc_3
SUCROSE_SYNTHASE_RXN_c_gc_2
MGII_ae_gc_2
aMAL_ae_gc_2
TRIOSEPISOMERIZATION_RXN_p_gc_2
aMAL_ae_gc_4
MGII_ae_gc_4
PRO_v_gc_Linker_1
K_total_pseudolinker_2
L_ASPARTATE_v_gc_Linker_2
RXN_1827_p_gc_2
Cl_v_gc_Linker_2
PEPCARBOX_RXN_c_gc_2
VAL_v_gc_Linker_1
Cl_c_gc_Linker_1
2KG_SUC_mc_gc_2
GLN_v_gc_Linker_2
WATER_vc_gc_2
CO2_ec_gc_2
GLY_v_gc_Linker_1
aMAL_ae_gc_3
MGII_ae_gc_3
TRP_v_gc_Linker_1
FRU_v_gc_Linker_1
GDPKIN_RXN_m_gc_2
3PGAREARR_RXN_c_gc_2
SUCROSE_total_pseudolinker_2
2_PERIOD_4_PERIOD_1_PERIOD_111_RXN_c_gc_2
MET_v_gc_Linker_1
K_ae_gc_1
MAL_ae_gc_1
STARCH_p_gc_Linker_2
MET_v_gc_Linker_2
CAII_ae_gc_3
K_ae_gc_3
SUCROSE_c_gc_Linker_2
K_v_gc_Linker_2
GAPOXNPHOSPHN_RXN_c_gc_2
CO2_mc_gc_2
CAII_ae_gc_4
K_ae_gc_4
ASPAMINOTRANS_RXN_c_gc_2
VAL_v_gc_Linker_2
ADENYL_KIN_RXN_m_gc_2
CYS_v_gc_Linker_2
2_KETOGLUTARATE_xc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
OAA_MAL_mc_gc_2
SULFATE_ae_gc_1
Cl_ae_gc_1
FRU_ae_gc_1
K_ae_gc_2
MAL_ae_gc_2
SUC_SO4_mc_gc_2
ASPAMINOTRANS_RXN_x_gc_2

In [20]:
# | export

white_constrained_wt_solution.to_csv(output_dir + "white_constrained_wt.csv")

#### Starch KO

In [21]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        white_constrained_starchko,
        white_constrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 20:45:57.564425
Running pFBA
Running FVA on 350 reactions using 4 processes
ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1

NITRATE_ae_gc_3
SUCROSE_SYNTHASE_RXN_c_gc_2
MGII_ae_gc_2
aMAL_ae_gc_2
TRIOSEPISOMERIZATION_RXN_p_gc_2
aMAL_ae_gc_4
MGII_ae_gc_4
PRO_v_gc_Linker_1
K_total_pseudolinker_2
L_ASPARTATE_v_gc_Linker_2
RXN_1827_p_gc_2
Cl_v_gc_Linker_2
PEPCARBOX_RXN_c_gc_2
VAL_v_gc_Linker_1
Cl_c_gc_Linker_1
2KG_SUC_mc_gc_2
GLN_v_gc_Linker_2
WATER_vc_gc_2
CO2_ec_gc_2
GLY_v_gc_Linker_1
aMAL_ae_gc_3
MGII_ae_gc_3
TRP_v_gc_Linker_1
FRU_v_gc_Linker_1
GDPKIN_RXN_m_gc_2
3PGAREARR_RXN_c_gc_2
SUCROSE_total_pseudolinker_2
2_PERIOD_4_PERIOD_1_PERIOD_111_RXN_c_gc_2
MET_v_gc_Linker_1
K_ae_gc_1
MAL_ae_gc_1
STARCH_p_gc_Linker_2
MET_v_gc_Linker_2
CAII_ae_gc_3
K_ae_gc_3
SUCROSE_c_gc_Linker_2
K_v_gc_Linker_2
GAPOXNPHOSPHN_RXN_c_gc_2
CO2_mc_gc_2
CAII_ae_gc_4
K_ae_gc_4
ASPAMINOTRANS_RXN_c_gc_2
VAL_v_gc_Linker_2
ADENYL_KIN_RXN_m_gc_2
CYS_v_gc_Linker_2
2_KETOGLUTARATE_xc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
OAA_MAL_mc_gc_2
SULFATE_ae_gc_1
Cl_ae_gc_1
FRU_ae_gc_1
K_ae_gc_2
MAL_ae_gc_2
SUC_SO4_mc_gc_2
ASPAMINOTRANS_RXN_x_gc_2

In [22]:
# | export

white_constrained_starchko_solution.to_csv(output_dir + "white_constrained_starchko.csv")

## Guard cell has no photosynthetic capacity

### Unconstrained ATPase

#### WT

In [23]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 20:53:15.383739
Running pFBA
Running FVA on 350 reactions using 4 processes
ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1

ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1
GAP_Pi_pc_gc_2
OAA_ISOCITRATE_mc_gc_2
TYR_v_gc_Linker_2
K_PROTON_cv_gc_2
SUC_FUM_mc_gc_2
SUCROSE_ae_gc_3
GLC_ae_gc_3
CI

In [24]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_unconstrained_wt.csv")

#### Starch KO

In [25]:
# | export
# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    (
        nops_unconstrained_starchko,
        nops_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 21:01:20.215898
Running pFBA
Running FVA on 350 reactions using 4 processes
ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1

ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1
GAP_Pi_pc_gc_2
OAA_ISOCITRATE_mc_gc_2
TYR_v_gc_Linker_2
K_PROTON_cv_gc_2
SUC_FUM_mc_gc_2
SUCROSE_ae_gc_3
GLC_ae_gc_3
CI

In [26]:
# | export

nops_unconstrained_starchko_solution.to_csv(output_dir + "nops_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [27]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 21:09:48.272563
Running pFBA
Running FVA on 350 reactions using 4 processes
ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1

NITRATE_ae_gc_3
SUCROSE_SYNTHASE_RXN_c_gc_2
MGII_ae_gc_2
aMAL_ae_gc_2
TRIOSEPISOMERIZATION_RXN_p_gc_2
aMAL_ae_gc_4
MGII_ae_gc_4
PRO_v_gc_Linker_1
K_total_pseudolinker_2
L_ASPARTATE_v_gc_Linker_2
RXN_1827_p_gc_2
Cl_v_gc_Linker_2
PEPCARBOX_RXN_c_gc_2
VAL_v_gc_Linker_1
Cl_c_gc_Linker_1
2KG_SUC_mc_gc_2
GLN_v_gc_Linker_2
WATER_vc_gc_2
CO2_ec_gc_2
GLY_v_gc_Linker_1
aMAL_ae_gc_3
MGII_ae_gc_3
TRP_v_gc_Linker_1
FRU_v_gc_Linker_1
GDPKIN_RXN_m_gc_2
3PGAREARR_RXN_c_gc_2
SUCROSE_total_pseudolinker_2
2_PERIOD_4_PERIOD_1_PERIOD_111_RXN_c_gc_2
MET_v_gc_Linker_1
K_ae_gc_1
MAL_ae_gc_1
STARCH_p_gc_Linker_2
MET_v_gc_Linker_2
CAII_ae_gc_3
K_ae_gc_3
SUCROSE_c_gc_Linker_2
K_v_gc_Linker_2
GAPOXNPHOSPHN_RXN_c_gc_2
CO2_mc_gc_2
CAII_ae_gc_4
K_ae_gc_4
ASPAMINOTRANS_RXN_c_gc_2
VAL_v_gc_Linker_2
ADENYL_KIN_RXN_m_gc_2
CYS_v_gc_Linker_2
2_KETOGLUTARATE_xc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
OAA_MAL_mc_gc_2
SULFATE_ae_gc_1
Cl_ae_gc_1
FRU_ae_gc_1
K_ae_gc_2
MAL_ae_gc_2
SUC_SO4_mc_gc_2
ASPAMINOTRANS_RXN_x_gc_2

In [28]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_constrained_wt.csv")

#### Starch KO

In [29]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        nops_unconstrained_starchko,
        nops_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2024-06-11 21:17:36.328048
Running pFBA
Running FVA on 350 reactions using 4 processes
ADENYL_KIN_RXN_c_gc_2
3PGAREARR_RXN_p_gc_2
MAL_SUC_mc_gc_2
Cl_total_pseudolinker_1
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
Cl_c_gc_Linker_2
Cl_total_pseudolinker_2
WATER_ae_gc_1
CAII_ae_gc_1
MALIC_NADP_RXN_c_gc_2
GLN_v_gc_Linker_1
PRO_v_gc_Linker_2
CIT_total_pseudolinker_1
X5P_Pi_pc_gc_2
GLC_total_pseudolinker_1
CIT_total_pseudolinker_2
MAL_CIT_mc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
PGLUCISOM_RXN_c_gc_2
GLC_total_pseudolinker_2
GAPOXNPHOSPHN_RXN_p_gc_2
GLY_v_gc_Linker_2
SER_v_gc_Linker_1
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
MAL_total_pseudolinker_1
SUCROSE_ae_gc_1
GLC_ae_gc_1
WATER_ae_gc_2
CAII_ae_gc_2
MAL_total_pseudolinker_2
Cl_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
GLYOHMETRANS_RXN_p_gc_2
SUCROSE_ae_gc_2
GLC_ae_gc_2
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
TYR_v_gc_Linker_1

NITRATE_ae_gc_3
SUCROSE_SYNTHASE_RXN_c_gc_2
MGII_ae_gc_2
aMAL_ae_gc_2
TRIOSEPISOMERIZATION_RXN_p_gc_2
aMAL_ae_gc_4
MGII_ae_gc_4
PRO_v_gc_Linker_1
K_total_pseudolinker_2
L_ASPARTATE_v_gc_Linker_2
RXN_1827_p_gc_2
Cl_v_gc_Linker_2
PEPCARBOX_RXN_c_gc_2
VAL_v_gc_Linker_1
Cl_c_gc_Linker_1
2KG_SUC_mc_gc_2
GLN_v_gc_Linker_2
WATER_vc_gc_2
CO2_ec_gc_2
GLY_v_gc_Linker_1
aMAL_ae_gc_3
MGII_ae_gc_3
TRP_v_gc_Linker_1
FRU_v_gc_Linker_1
GDPKIN_RXN_m_gc_2
3PGAREARR_RXN_c_gc_2
SUCROSE_total_pseudolinker_2
2_PERIOD_4_PERIOD_1_PERIOD_111_RXN_c_gc_2
MET_v_gc_Linker_1
K_ae_gc_1
MAL_ae_gc_1
STARCH_p_gc_Linker_2
MET_v_gc_Linker_2
CAII_ae_gc_3
K_ae_gc_3
SUCROSE_c_gc_Linker_2
K_v_gc_Linker_2
GAPOXNPHOSPHN_RXN_c_gc_2
CO2_mc_gc_2
CAII_ae_gc_4
K_ae_gc_4
ASPAMINOTRANS_RXN_c_gc_2
VAL_v_gc_Linker_2
ADENYL_KIN_RXN_m_gc_2
CYS_v_gc_Linker_2
2_KETOGLUTARATE_xc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
OAA_MAL_mc_gc_2
SULFATE_ae_gc_1
Cl_ae_gc_1
FRU_ae_gc_1
K_ae_gc_2
MAL_ae_gc_2
SUC_SO4_mc_gc_2
ASPAMINOTRANS_RXN_x_gc_2

In [30]:
# | export

nops_unconstrained_starchko_solution.to_csv(output_dir + "nops_constrained_starchko.csv")

## Scan GC photon influx to look at switch of glycolysis

In [31]:
P_gc, P_me = arabidopsis_supermodel.get_photons(PPFD=PPFD, printouts=False)

for i, p in enumerate([0, P_me, P_me, 0], start=1):
    arabidopsis_supermodel.fba_model.reactions.get_by_id("Photon_tx_me_" + str(i)).bounds = 0, p

with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    scan_df = pd.DataFrame()
    for photon in range(PPFD+1):
        P_gc, P_me = arabidopsis_supermodel.get_photons(PPFD=photon, printouts=False)
        for i, p in enumerate([0, P_gc, P_gc, 0], start=1):
            m.reactions.get_by_id("Photon_tx_gc_" + str(i)).bounds = 0, p
        cobra_model, solution = mmon_gcm.solving.pfba_Weighted(m, objective="Phloem_tx_overall")
        scan_df[photon] = solution.fluxes
        print(photon, solution.fluxes['Photon_tx_gc_2'], solution.fluxes['Photon_tx_me_2'])
scan_df.to_csv("../outputs/scans/gc_photon_scan.csv")

0 0.0 485.98162812525
1 0.000122479164998293 485.98162812525
2 0.000244958329996587 485.98162812525
3 0.00036743749499488 485.98162812525
4 0.000489916659993173 485.98162812525
5 0.000612395824991467 485.98162812525
6 0.00073487498998976 485.98162812525
7 0.000857354154988053 485.98162812525
8 0.000979833319986346 485.98162812525
9 0.00110231248498464 485.98162812525
10 0.00122479164998293 485.98162812525
11 0.00134727081498123 485.98162812525
12 0.00146974997997952 485.98162812525
13 0.00159222914497781 485.98162812525
14 0.00171470830997611 485.98162812525
15 0.0018371874749744 485.98162812525
16 0.00195966663997269 485.98162812525
17 0.00208214580497099 485.98162812525
18 0.00220462496996928 485.98162812525
19 0.00232710413496757 485.98162812525
20 0.00244958329996587 485.98162812525
21 0.00257206246496416 485.98162812525
22 0.00269454162996245 485.98162812525
23 0.00281702079496075 485.98162812525
24 0.00293949995995904 485.98162812525
25 0.00306197912495733 485.98162812525
26 0.00

/tmp/ipykernel_4680/4142017668.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


100 0.0122479164998293 485.98162812525


/tmp/ipykernel_4680/4142017668.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


101 0.0123703956648276 485.98162812525


...

148 0.0181269164197474 485.98162812525


/tmp/ipykernel_4680/4142017668.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


149 0.0182493955847457 485.98162812525


/tmp/ipykernel_4680/4142017668.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


150 0.018371874749744 485.98162812525


## Scan GC & ME photosynthesis (photon influx) during Opening

Increase upper bound of photon influx in GC & ME during blue light

In [32]:
P_gc_day, P_me_day = arabidopsis_supermodel.get_photons(PPFD=PPFD, printouts=False)

with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    scan_df = pd.DataFrame()
    for photon in range(PPFD+1):
        P_gc_open, P_me_open = arabidopsis_supermodel.get_photons(PPFD=photon, printouts=False)
        for i, p in enumerate([0, P_me_open, P_me_day, 0], start=1):
            m.reactions.get_by_id("Photon_tx_me_" + str(i)).bounds = 0, p
        for i, p in enumerate([0, P_gc_open, P_gc_day, 0], start=1):
            m.reactions.get_by_id("Photon_tx_gc_" + str(i)).bounds = 0, p
        cobra_model, solution = mmon_gcm.solving.pfba_Weighted(m, objective="Phloem_tx_overall")
        scan_df[photon] = solution.fluxes
        print(photon, solution.fluxes['Photon_tx_gc_2'], solution.fluxes['Photon_tx_me_2'],
              solution.fluxes['Photon_tx_gc_3'], solution.fluxes['Photon_tx_me_3'])
scan_df.to_csv("../outputs/scans/opening_photon_scan.csv")

0 0.0 0.0 0.018371874749744 485.98162812525
1 0.000122479164998293 3.239877520835 0.018371874749744 485.98162812525
2 0.000244958329996587 6.47975504167 0.018371874749744 485.98162812525
3 0.00036743749499488 9.71963256250501 0.018371874749744 485.98162812525
4 0.000489916659993173 12.95951008334 0.018371874749744 485.98162812525
5 0.000612395824991467 16.199387604175 0.018371874749744 485.98162812525
6 0.00073487498998976 19.43926512501 0.018371874749744 485.98162812525
7 0.000857354154988053 22.679142645845 0.018371874749744 485.98162812525
8 0.000979833319986346 25.91902016668 0.018371874749744 485.98162812525
9 0.00110231248498464 29.158897687515 0.018371874749744 485.98162812525
10 0.00122479164998293 32.39877520835 0.018371874749744 485.98162812525
11 0.00134727081498123 35.638652729185 0.018371874749744 485.98162812525
12 0.00146974997997952 38.87853025002 0.018371874749744 485.98162812525
13 0.00159222914497781 42.118407770855 0.018371874749744 485.98162812525
14 0.001714708309

/tmp/ipykernel_4680/4218495784.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


100 0.0122479164998293 323.9877520835 0.018371874749744 485.98162812525


/tmp/ipykernel_4680/4218495784.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


101 0.0123703956648276 327.227629604335 0.018371874749744 485.98162812525


...

148 0.0181269164197474 479.50187308358 0.018371874749744 485.98162812525


/tmp/ipykernel_4680/4218495784.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


149 0.0182493955847457 482.741750604415 0.018371874749744 485.98162812525


/tmp/ipykernel_4680/4218495784.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


150 0.018371874749744 485.98162812525 0.018371874749744 485.98162812525
